In [28]:

import math, json, re, ccxt
from pprint import pformat
wex = ccxt.wex()
data_for_graph = {}
data = wex.fetch_tickers()
for key in data:
    data_for_graph.update({key: data[key]['bid']})
ticker_list = list(data_for_graph.keys())
ticker_list2 = []
for key in ticker_list:
    ticker_list2.append("{1}/{0}".format(*key.split('/'))) #обратные тикеры

for key in range(0, len(ticker_list)):
    #обратные котировки
    data_for_graph.update({ticker_list2[key]: 1 / (data[ticker_list[key]]['ask'])})

print(pformat(data_for_graph))


{'BCH/BCHET': 1.51285930408472,
 'BCH/BTC': 0.1119,
 'BCH/DASH': 2.434,
 'BCH/ETH': 1.946,
 'BCH/EUR': 738.12,
 'BCH/LTC': 6.299,
 'BCH/RUR': 51419.051,
 'BCH/USD': 900.0,
 'BCH/ZEC': 4.104,
 'BCHET/BCH': 0.66,
 'BTC/BCH': 8.904719501335709,
 'BTC/BTCET': 1.5432098765432098,
 'BTC/DASH': 21.68726957276079,
 'BTC/ETH': 17.289073305670815,
 'BTC/EUR': 6556.05324,
 'BTC/LTC': 56.306306306306304,
 'BTC/NMC': 4761.9047619047615,
 'BTC/NVC': 2127.6595744680853,
 'BTC/PPC': 4347.826086956522,
 'BTC/RUR': 459999.99999,
 'BTC/USD': 8023.02,
 'BTC/ZEC': 36.496350364963504,
 'BTCET/BTC': 0.645,
 'DASH/BCH': 0.40866366979975477,
 'DASH/BTC': 0.04583,
 'DASH/DSHET': 1.3888888888888888,
 'DASH/ETH': 0.795,
 'DASH/EUR': 302.314,
 'DASH/LTC': 2.587,
 'DASH/RUR': 21087.963,
 'DASH/USD': 368.94779,
 'DASH/ZEC': 1.682,
 'DSHET/DASH': 0.715,
 'ETH/BCH': 0.5102040816326531,
 'ETH/BTC': 0.05753,
 'ETH/DASH': 1.2484394506866416,
 'ETH/ETHET': 1.6313213703099512,
 'ETH/EUR': 377.5243,
 'ETH/LTC': 3.236,
 'ETH

Создаем граф в виде списка смежности для каждой ноды - тикера криптовалюты, 
котировки переводим в логарифмический вид.

In [29]:
graph = {}
jsrates = data_for_graph

pattern = re.compile("([A-Z]{3,5})/([A-Z]{3,5})")

try:
    for key in jsrates:
        matches = pattern.match(key)

        conversion_rate = math.log(float(jsrates[key]))
        
        from_rate = matches.group(1).encode('ascii', 'ignore')

        to_rate = matches.group(2).encode('ascii', 'ignore')

        if from_rate != to_rate:
            if from_rate not in graph:
                graph[from_rate] = {}
            graph[from_rate][to_rate] = float(conversion_rate)

except AttributeError:
    print('Strange rate')
    pass

print(pformat(graph))
# воплощение алгоритма отсюда
#  https://gist.github.com/joninvski/701720


{b'BCH': {b'BCHET': 0.4140014391304507,
          b'BTC': -2.1901496636642577,
          b'DASH': 0.8895359945653355,
          b'ETH': 0.6657759837638133,
          b'EUR': 6.604106413008277,
          b'LTC': 1.840390890639792,
          b'RUR': 10.847764024795815,
          b'USD': 6.802394763324311,
          b'ZEC': 1.4119621078684685},
 b'BCHET': {b'BCH': -0.4155154439616658},
 b'BTC': {b'BCH': 2.18658141723774,
          b'BTCET': 0.4338645826298623,
          b'DASH': 3.076725432763523,
          b'ETH': 2.8500747011316276,
          b'EUR': 8.788144060636116,
          b'LTC': 4.030806541418113,
          b'NMC': 8.468403027246806,
          b'NVC': 7.66277786326017,
          b'PPC': 8.377431249041079,
          b'RUR': 13.038981768443538,
          b'USD': 8.990070188583653,
          b'ZEC': 3.5972122655881127},
 b'BTCET': {b'BTC': -0.43850496218636453},
 b'DASH': {b'BCH': -0.8948627844605347,
           b'BTC': -3.082816380461104,
           b'DSHET': 0.32850406697203605,


In [30]:


# Step 1: For each node prepare the destination and predecessor
def initialize(graph, source):
    d = {}  # Stands for destination
    p = {}  # Stands for predecessor
    for node in graph:
        d[node] = float('Inf')  # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0  # For the source we know how to reach
    return d, p


def relax(node, neighbour, graph, d, p):
    # If the distance between the node and the neighbour is lower than the one I have now
    if d[neighbour] > d[node] + graph[node][neighbour]:
        # Record this lower distance
        d[neighbour] = d[node] + graph[node][neighbour]
        p[neighbour] = node


def retrace_negative_loop(p, start):
    arbitrageLoop = [start]
    next_node = start
    while True:
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
            return arbitrageLoop


def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph) - 1):  # Run this until is converges
        for u in graph:
            for v in graph[u]:  # For each neighbour of u
                relax(u, v, graph, d, p)  # Lets relax it

    # Step 3: check for negative-weight cycles
    for u in graph:
        for v in graph[u]:
            if d[v] < d[u] + graph[u][v]:
                return (retrace_negative_loop(p, source))
    return None


In [36]:
 paths = []
 for key in graph:
    
    path = bellman_ford(graph, key)

    if path not in paths and not None:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate*0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")

 

Starting with 100 in b'BTC'
b'BTC' to b'PPC' at 4347.826087 = 433913.043478
b'PPC' to b'BTC' at 0.000220 = 95.269948
Starting with 100 in b'PPC'
b'PPC' to b'BTC' at 0.000220 = 0.021956
b'BTC' to b'PPC' at 4347.826087 = 95.269948


